## Import Pacotes

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
from datetime import datetime
from xgboost import plot_importance
import matplotlib.pyplot as plt

## Funcoes

In [ ]:
def limites_outlier(X_train, label = None):
    if label == 'REDE':
        LI = (X_train[label]).quantile(0.05)
        LS = (X_train[label]).quantile(0.95)

    elif label == 'VALOR_TOTAL':
        Q1 = (X_train[label]).quantile(0.25)
        Q3 = (X_train[label]).quantile(0.75)
        IQR = Q3 - Q1
        LS = Q3 + 1.5*(IQR)
        LI = Q1 - 1.5*(IQR)
    return(LI,LS)

def create_features(X_train, LI, LS, label=None):
    X_train = X_train.copy()
    y_train = X_train[label]
    X_train['DIA_SEMANA'] = X_train['DATA'].dt.dayofweek
    X_train['TRIMESTRE'] = X_train['DATA'].dt.quarter
    X_train['MES'] = X_train['DATA'].dt.month
    X_train['ANO'] = X_train['DATA'].dt.year
    X_train['DIA_ANO'] = X_train['DATA'].dt.dayofyear
    X_train['DIA_MES'] = X_train['DATA'].dt.day
    X_train['SEMANA_ANO'] = X_train['DATA'].dt.isocalendar().week.astype(int)
    X_train['TER'] = (X_train['DATA'].dt.dayofweek == 1).astype(int)
    X_train['QUA'] = (X_train['DATA'].dt.dayofweek == 2).astype(int)
    X_train['QUI'] = (X_train['DATA'].dt.dayofweek == 3).astype(int)
    X_train['SEX'] = (X_train['DATA'].dt.dayofweek == 4).astype(int)
    X_train['SAB'] = (X_train['DATA'].dt.dayofweek == 5).astype(int)
    X_train['DOM'] = (X_train['DATA'].dt.dayofweek == 6).astype(int)
    X_train['SEG'] = (X_train['DATA'].dt.dayofweek == 0).astype(int)
    X_train["OUTLIER"] = np.where(y_train>LS,1,np.where(y_train<LI,-1,0))
    X_train['DECORRIDO'] = (pd.to_datetime(max(X_train['DATA'])) - pd.to_datetime(X_train['DATA'])).dt.days

    if label == 'REDE':
        X_train = X_train[['DIA_SEMANA','TRIMESTRE','MES','ANO','DIA_ANO','DIA_MES','SEMANA_ANO','FERIADO','ULTIMO_DIA',
                'PRIMEIRO_DIA','SEMANA_MES','VESPERA','BRIDGE','ANO_NOVO','CARNAVAL1','CARNAVAL2','CARNAVAL3',
                'PAIXAO_CRISTO','PASCOA','TIRADENTES','DIA_DO_TRABALHO','VESPERA_MAES','DIA_DAS_MAES',
                'CORPUS_CHRISTI','DIA_DOS_PAIS','INDEPENDENCIA','APARECIDA','FINADOS','PROC_REPUBLICA',
                'BLACK_THURSDAY','BLACK_FRIDAY','VESPERA_NATAL','NATAL','VESPERA_ANO_NOVO','DECORRIDO','OUTLIER',
                'POS_SEXTA','VESPERA_SEXTA','SEX','SAB','DOM']]
    elif label == 'VALOR_TOTAL':
        X_train = X_train[['DIA_SEMANA','TRIMESTRE','MES','ANO','DIA_ANO','DIA_MES','SEMANA_ANO','FERIADO','ULTIMO_DIA',
                'PRIMEIRO_DIA','SEMANA_MES','BRIDGE','VESPERA','ANO_NOVO','CARNAVAL1','CARNAVAL2','CARNAVAL3',
                'PAIXAO_CRISTO','PASCOA','TIRADENTES','DIA_DO_TRABALHO','VESPERA_MAES','DIA_DAS_MAES',
                'CORPUS_CHRISTI','DIA_DOS_PAIS','INDEPENDENCIA','APARECIDA','FINADOS','PROC_REPUBLICA',
                'BLACK_THURSDAY','BLACK_FRIDAY','VESPERA_NATAL','NATAL','VESPERA_ANO_NOVO','OUTLIER',
                'SEX','SAB','DOM','SEMANA_PASCOA','POS_SEXTA','VESPERA_SEXTA']]
    else:
        raise ValueError('Invalid label')

    

    return X_train,y_train

def previsao(X_train,X_test,parameters,scoring,label=None,Metric = 'MAE'):
    LI,LS = limites_outlier(X_train,label)
    X_train,y_train = create_features(X_train,LI,LS,label)
    X_test,y_test = create_features(X_test,LI,LS,label)
    
    if label == 'REDE':
        X_test['OUTLIER'] = 0
        grid_search = GridSearchCV(xgb.XGBRegressor(), parameters, scoring=scoring, refit=Metric, cv=5)
        grid_search.fit(X_train, y_train)
        Previsto = (grid_search.best_estimator_).predict(X_test)
        importance = plot_importance(grid_search.best_estimator_, height=0.5)
        plt.savefig('importance_plot_REDE.png')

    elif label == 'VALOR_TOTAL':
        grid_search = GridSearchCV(xgb.XGBRegressor(), parameters, scoring=scoring, refit=Metric, cv=5)
        grid_search.fit(X_train, y_train)
        Previsto = (grid_search.best_estimator_).predict(X_test)
        importance = plot_importance(grid_search.best_estimator_, height=0.5)
        plt.savefig('importance_plot_VAREJO.png')

    return Previsto, importance

## Leitura dos dados

In [ ]:
dfpath = r"\\...\WORK_BASE_XGBOOST.csv"
df = pd.read_csv(dfpath, sep=',')
df['DATA'] = pd.to_datetime( df['DATA'], format= "%d-%m-%y")
df_train = (df.loc[df['DATA'] < pd.to_datetime(datetime.today().replace(hour=0, minute=0, second=0, microsecond=0))])
df_pred = (df.loc[df['DATA'] >= pd.to_datetime(datetime.today().replace(hour=0, minute=0, second=0, microsecond=0))])

#Ultimo mês sera usado para validacao
df = df.loc[df['DATA'] <= '2023-06-30'] 
df_train = (df.loc[df['DATA'] < '2023-06-01'])
df_pred = (df.loc[df['DATA'] >= '2023-06-01'])

## Parametros GridSearch

In [ ]:
parameters = {
    'n_estimators': [100, 500, 1000],
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.3],
    'objective': ['reg:squarederror','reg:tweedie']
}
parameters_REDE = {
    'n_estimators': [100, 500, 1000],
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.3],
    'objective': ['reg:squarederror']
}

#### Metricas Escolha do Melhor Modelo

In [ ]:
scoring = {
    'MAE': make_scorer(mean_absolute_error, greater_is_better=False),
    'MSE': make_scorer(mean_squared_error, greater_is_better=False),
    'RMSE': make_scorer(lambda y_true, y_pred: -np.sqrt(mean_squared_error(y_true, y_pred))),
    'R2': make_scorer(r2_score)
}

## Previsao

In [ ]:
Previsao_varejo, importancias_varejo = previsao(df_train,df_pred,parameters,scoring,label='VALOR_TOTAL',Metric= 'MAE')
Previsao_REDE, importancias_REDE = previsao(df_train,df_pred,parameters_REDE,scoring,label='VLR_REDE',Metric= 'R2')
Previsao_dia = pd.DataFrame({'DATA': df_pred['DATA'],'PREVISAO': Previsao_varejo, 'PREVISAO_REDE': Previsao_REDE})
Previsao_dia.tail()

## Metricas para Varejo

In [ ]:
MAE = mean_absolute_error(df_pred['VALOR_TOTAL'],Previsao_varejo)
MSE = mean_squared_error(df_pred['VALOR_TOTAL'],Previsao_varejo)
R2 = r2_score(df_pred['VALOR_TOTAL'],Previsao_varejo)

print('MAE:', MAE)
print('MSE:', MSE)
print('R2:', R2)

## Metricas para REDE

In [ ]:
MAE = mean_absolute_error(df_pred['VLR_REDE'],Previsao_REDE)
MSE = mean_squared_error(df_pred['VLR_REDE'],Previsao_REDE)
R2 = r2_score(df_pred['VLR_REDE'],Previsao_REDE)

print('MAE:', MAE)
print('MSE:', MSE)
print('R2:', R2)